In [24]:
!pip install --upgrade transformers==4.41.1

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/9.1 MB ? eta -:--:--
   --------------------------- ------------ 6.3/9.1 MB 38.9 MB/s eta 0:00:01
   ---------------------------------------- 9.1/9.1 MB 24.3 MB/s  0:00:00
Failed to build tokenizers


  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [106 lines of output]
      Running `maturin pep517 build-wheel -i C:\Users\luluk\AppData\Local\Programs\Python\Python313\python.exe --compatibility off`
      âš ï¸\x8f  Warning: `project.version` field is required in pyproject.toml unless it is present in the `project.dynamic` list
      ðŸ\x8d¹ Building a mixed python/rust project
      ðŸ”— Found pyo3 bindings
      ðŸ\x90\x8d Found CPython 3.13 at C:\Users\luluk\AppData\Local\Programs\Python\Python313\python.exe
      ðŸ“¡ Using build options features, bindings from pyproject.toml
         Compiling proc-macro2 v1.0.81
         Compiling unicode-ident v1.0.12
         Compiling autocfg v1.2.0
         Compiling target-lexicon v0.12.14
         Compiling windows_x86_64_msvc v0.52.5
         Compiling once_cell v1.19.0
         Compiling libc v0.2.153
         Compiling cfg-if v1.0.0
         C

In [1]:
import torch
import numpy as np
import pandas as pd

In [2]:
print(torch.version.cuda)

if torch.cuda.is_available():
    print("GPU is available.")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current GPU name: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU available. PyTorch will use CPU.")

12.6
GPU is available.
Number of GPUs: 1
Current GPU name: NVIDIA GeForce GTX 1650


In [3]:
df = pd.read_csv("hf://datasets/jojo-ai-mst/Roleplay-Indonesian/GPTeacher RolePlay Simple Deduped - Indonesian.csv")

In [4]:
df

,instruction,input,response,petunjuk,masukan,tanggapan
0,Imagine you are a detective trying to solve a ...,NaN,The scene of the crime is a dimly lit study in...,Bayangkan Anda adalah seorang detektif yang me...,#VALUE!,TKP adalah sebuah ruang belajar remang-remang ...
1,You are an alien diplomat visiting Earth for t...,NaN,My initial impressions of Earth are dominated ...,Anda adalah seorang diplomat asing yang mengun...,#VALUE!,Kesan awal saya terhadap Bumi didominasi oleh ...
2,Imagine you are an ambassador from a planet ca...,NaN,"Greetings, esteemed Earth leaders. I am the am...",Bayangkan Anda adalah seorang duta besar dari ...,#VALUE!,"Salam, para pemimpin Bumi yang terhormat. Saya..."
3,Pretend you are a detective in the 1920s inter...,The witness claims to have seen the suspect fl...,"1. Can you describe the suspect's appearance, ...",Anggaplah Anda adalah seorang detektif di tahu...,Saksi mengaku melihat tersangka melarikan diri...,1. Bisakah Anda menjelaskan penampilan tersang...
4,"You are a time traveler from the 30th century,...",NaN,1. Fossil fuel-based transportation: In the 30...,"Anda adalah penjelajah waktu dari abad ke-30, ...",#VALUE!,1. Transportasi berbasis bahan bakar fosil: Pa...
...,...,...,...,...,...,...
1918,You are the president of a fictional country c...,NaN,"My dear citizens of Estoria, I stand before yo...",Anda adalah presiden negara fiksi bernama Esto...,#VALUE!,"Warga Estoria yang terkasih, saya berdiri di h..."
1919,You are renowned historian Dr. Isabella Wright...,NaN,The invention of the printing press revolution...,Anda adalah sejarawan terkenal Dr. Isabella Wr...,#VALUE!,Penemuan mesin cetak merevolusi sejarah manusi...
1920,"Role play as Dolly Parton, a famous singer-son...",NaN,My inspiration for the Imagination Library pro...,"Bermain peran sebagai Dolly Parton, penyanyi-p...",#VALUE!,Inspirasi saya untuk proyek Perpustakaan Imaji...
1921,You are a renowned historian specializing in a...,NaN,"The Sumerian culture, originating in ancient M...",Anda adalah seorang sejarawan terkenal yang be...,#VALUE!,"Kebudayaan Sumeria, yang berasal dari Mesopota..."


In [5]:
df["input"].isna().sum()

np.int64(1480)

In [6]:
try:
    # --- 1. Identify all three columns we need ---
    indonesian_response_col = df.columns[5] # The 6th column
    
    # Select the three Indonesian columns
    clean_df = df[['petunjuk', 'masukan', indonesian_response_col]].copy()

    # --- 2. Rename columns for clarity ---
    clean_df.rename(columns={
        'petunjuk': 'instruksi', # Renamed to avoid confusion with final 'prompt'
        'masukan': 'konteks',
        indonesian_response_col: 'respons'
    }, inplace=True)

    # --- 3. Clean the 'konteks' (masukan) column ---
    # Replace the text "#VALUE!" with a proper null value (np.nan)
    clean_df['konteks'] = clean_df['konteks'].replace("#VALUE!", np.nan)

    # --- 4. Define our prompt-building function ---
    def create_full_prompt(row):
        instruksi = row['instruksi']
        konteks = row['konteks']
        
        # Check if kontek is NaN, None, or an empty string
        if pd.isna(konteks) or konteks.strip() == "":
            return instruksi # Use instruction only
        else:
            # Combine them using our template
            return f"{instruksi}\n\n{konteks}"

    # --- 5. Apply the function to create the final 'prompt' column ---
    print("Combining 'instruksi' and 'konteks' into a single 'prompt'...")
    clean_df['prompt'] = clean_df.apply(create_full_prompt, axis=1)
    print("...Done!\n")

    # --- 6. Create our final DataFrame ---
    # We only need the new 'prompt' and the 'respons'
    final_df = clean_df[['prompt', 'respons']].copy()

    # Drop any rows where the final prompt or response is missing
    final_df.dropna(inplace=True)

    # --- 7. Show the results ---
    print("--- Final Cleaned DataFrame (First 5 Rows) ---")
    print(final_df.head())
    print("\n")
    
    # --- 8. Show an example of a combined prompt ---
    # Find a row where 'konteks' was NOT empty to prove it worked
    combined_example_row = clean_df[clean_df['konteks'].notnull()]
    
    if not combined_example_row.empty:
        print("--- Example of a Combined Prompt (from row 0) ---")
        print(combined_example_row.iloc[0]['prompt'])
    else:
        print("Could not find a combined prompt example in the first batch.")

except Exception as e:
    print(f"An error occurred: {e}")

Combining 'instruksi' and 'konteks' into a single 'prompt'...
...Done!

--- Final Cleaned DataFrame (First 5 Rows) ---
                                              prompt  \
0  Bayangkan Anda adalah seorang detektif yang me...   
1  Anda adalah seorang diplomat asing yang mengun...   
2  Bayangkan Anda adalah seorang duta besar dari ...   
3  Anggaplah Anda adalah seorang detektif di tahu...   
4  Anda adalah penjelajah waktu dari abad ke-30, ...   

                                             respons  
0  TKP adalah sebuah ruang belajar remang-remang ...  
1  Kesan awal saya terhadap Bumi didominasi oleh ...  
2  Salam, para pemimpin Bumi yang terhormat. Saya...  
3  1. Bisakah Anda menjelaskan penampilan tersang...  
4  1. Transportasi berbasis bahan bakar fosil: Pa...  


--- Example of a Combined Prompt (from row 0) ---
Anggaplah Anda adalah seorang detektif di tahun 1920-an yang mewawancarai seorang saksi kejahatan. Ajukan 5 pertanyaan untuk mengumpulkan informasi tentang tersan

In [7]:
clean_df

,instruksi,konteks,respons,prompt
0,Bayangkan Anda adalah seorang detektif yang me...,NaN,TKP adalah sebuah ruang belajar remang-remang ...,Bayangkan Anda adalah seorang detektif yang me...
1,Anda adalah seorang diplomat asing yang mengun...,NaN,Kesan awal saya terhadap Bumi didominasi oleh ...,Anda adalah seorang diplomat asing yang mengun...
2,Bayangkan Anda adalah seorang duta besar dari ...,NaN,"Salam, para pemimpin Bumi yang terhormat. Saya...",Bayangkan Anda adalah seorang duta besar dari ...
3,Anggaplah Anda adalah seorang detektif di tahu...,Saksi mengaku melihat tersangka melarikan diri...,1. Bisakah Anda menjelaskan penampilan tersang...,Anggaplah Anda adalah seorang detektif di tahu...
4,"Anda adalah penjelajah waktu dari abad ke-30, ...",NaN,1. Transportasi berbasis bahan bakar fosil: Pa...,"Anda adalah penjelajah waktu dari abad ke-30, ..."
...,...,...,...,...
1918,Anda adalah presiden negara fiksi bernama Esto...,NaN,"Warga Estoria yang terkasih, saya berdiri di h...",Anda adalah presiden negara fiksi bernama Esto...
1919,Anda adalah sejarawan terkenal Dr. Isabella Wr...,NaN,Penemuan mesin cetak merevolusi sejarah manusi...,Anda adalah sejarawan terkenal Dr. Isabella Wr...
1920,"Bermain peran sebagai Dolly Parton, penyanyi-p...",NaN,Inspirasi saya untuk proyek Perpustakaan Imaji...,"Bermain peran sebagai Dolly Parton, penyanyi-p..."
1921,Anda adalah seorang sejarawan terkenal yang be...,NaN,"Kebudayaan Sumeria, yang berasal dari Mesopota...",Anda adalah seorang sejarawan terkenal yang be...


In [8]:
for i in range(3):
    print(f'prompt:\n{clean_df["prompt"][i]}\n')
    print(f'respons:\n{clean_df["respons"][i]}\n\n')

prompt:
Bayangkan Anda adalah seorang detektif yang mencoba memecahkan misteri pembunuhan di kota fiksi era Victoria. Jelaskan TKP, tersangka utama, dan rencana Anda untuk menyelidiki kasus tersebut.

respons:
TKP adalah sebuah ruang belajar remang-remang di dalam sebuah rumah megah era Victoria. Ada meja kayu ek besar di tengah ruangan, dan almarhum, Lord Barrington, terbaring lemas di atasnya dengan luka tembak di dada. Kaca jendela yang pecah menandakan kemungkinan masuknya penyerang. Tiga tersangka utama adalah: 1) Lady Barrington, istri korban, yang semakin sering berselisih dengan suaminya; 2) Randall, kepala pelayan setia keluarga, yang baru-baru ini mengetahui bahwa dia akan digantikan; dan 3) Edgar, sepupu jauh, yang mengharapkan warisan besar jika Lord Barrington meninggal. Rencana saya untuk menyelidiki kasus ini meliputi: mewawancarai setiap tersangka tentang keberadaan mereka selama kejahatan terjadi; memeriksa jendela yang pecah untuk mencari sidik jari atau tanda-tanda m

In [12]:
import torch
from transformers import GPT2Tokenizer

MODEL_NAME = "cahya/gpt2-small-indonesian-522M"

print(f"Loading tokenizer: {MODEL_NAME}")

# 1. Load the tokenizer.
# We use use_fast=False to load the Python-based "slow" tokenizer.
# This avoids a known bug in new 'transformers' versions that
# incorrectly look for a 'additional_chat_templates' file.
tokenizer = GPT2Tokenizer.from_pretrained(
    MODEL_NAME, 
    use_fast=False
)
print("...Tokenizer loaded.")

# 2. Define and add special tokens.
# The model's existing End-of-Sequence token is '<|endoftext|>' (ID 0).
# We add a new, dedicated token for padding.
special_tokens_dict = {
    'eos_token': '<|endoftext|>',
    'pad_token': '<|pad|>'
}

num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print(f"Added {num_added_toks} new special token(s).")

# 3. Store the final vocabulary size.
# We will need this number later when we build our model's embedding layer.
VOCAB_SIZE = len(tokenizer)
print(f"New vocabulary size: {VOCAB_SIZE}")


# 4. --- Real Tokenizer Test ---
print("\n--- Tokenizer Test ---")

# Use a real example from our roleplay dataset
prompt = "Anda adalah seorang detektif."
response = "Tentu, apa kasusnya?"

# This is the final format our model will be trained on:
# [PROMPT_TOKENS] + [RESPONSE_TOKENS] + [EOS_TOKEN]
full_text = prompt + response + tokenizer.eos_token

print(f"Original Text:\n{full_text}")

# Tokenize the text into a list of numbers
input_ids = tokenizer(full_text)['input_ids']

print(f"\nTokenized IDs:\n{input_ids}")
print(f"Token count: {len(input_ids)}")

# Decode the numbers back to text to verify
decoded_text = tokenizer.decode(input_ids)
print(f"\nDecoded Text:\n{decoded_text}")


# 5. --- Special Token Verification ---
print("\n--- Special Token Verification ---")
eos_id = tokenizer.eos_token_id
pad_id = tokenizer.pad_token_id

print(f"EOS token: '{tokenizer.eos_token}' -> ID: {eos_id}")
print(f"PAD token: '{tokenizer.pad_token}' -> ID: {pad_id}")

# This is the most critical check:
# We must ensure the PAD token and EOS token have different IDs.
assert eos_id != pad_id
print("\nSuccess! EOS and PAD tokens have different IDs.")

Loading tokenizer: cahya/gpt2-small-indonesian-522M
...Tokenizer loaded.
Added 1 new special token(s).
New vocabulary size: 50258

--- Tokenizer Test ---
Original Text:
Anda adalah seorang detektif.Tentu, apa kasusnya?<|endoftext|>

Tokenized IDs:
[20976, 360, 604, 17645, 14, 52, 1032, 12, 1985, 40326, 31, 0]
Token count: 12

Decoded Text:
Anda adalah seorang detektif.Tentu, apa kasusnya?<|endoftext|>

--- Special Token Verification ---
EOS token: '<|endoftext|>' -> ID: 0
PAD token: '<|pad|>' -> ID: 50257

Success! EOS and PAD tokens have different IDs.


In [34]:
class RoleplayDataset(Dataset):
    """
    A robust PyTorch Dataset for instruction-following.
    
    It tokenizes the prompt and response separately to ensure
    the mask is always perfectly aligned.
    """
    def __init__(self, df, tokenizer):
        self.tokenizer = tokenizer
        self.prompts = df['prompt'].tolist()
        self.responses = df['respons'].tolist()
        
        self.eos_token = tokenizer.eos_token
        self.eos_token_id = tokenizer.eos_token_id

    def __len__(self):
        return len(self.prompts)

    def __getitem__(self, idx):
        prompt = self.prompts[idx]
        response = self.responses[idx]

        # 1. --- Create and tokenize the two parts separately ---
        
        # Prompt part: Includes "User:", the prompt, and "Model: "
        # We add a space *after* "Model:" to signal the start of the response.
        prompt_str = f"User: {prompt}\n\nModel: "
        
        # Response part: Includes the response and the EOS token
        response_str = f"{response}{self.eos_token}"

        # 2. Tokenize both parts
        prompt_tokens = self.tokenizer(
            prompt_str, add_special_tokens=False
        )['input_ids']
        
        response_tokens = self.tokenizer(
            response_str, add_special_tokens=False
        )['input_ids']
        
        # 3. Combine them into the final tensors
        input_ids = torch.tensor(prompt_tokens + response_tokens, dtype=torch.long)
        labels = input_ids.clone()
        
        # 4. Apply the mask
        # The mask length is simply the length of our prompt_tokens
        prompt_token_len = len(prompt_tokens)
        labels[:prompt_token_len] = -100
        
        return input_ids, labels

# --- Test the NEW Dataset ---
print("--- New (Robust) Dataset Test ---")

# Use your real 'final_df' here, not the dummy one
train_dataset = RoleplayDataset(clean_df, tokenizer)

# 2. Get the first sample (index 0)
input_ids, labels = train_dataset[0]

print(f"Sample 0:\n")
print(f"Input IDs:\n{input_ids}")
print(f"\nLabels (with -100 mask):\n{labels}")

print("\n--- Decoded Verification ---")
decoded_input = tokenizer.decode(input_ids, skip_special_tokens=False)
print(f"\nDecoded Input:\n'{decoded_input}'")

labels_to_decode = labels[labels != -100]
decoded_labels = tokenizer.decode(labels_to_decode, skip_special_tokens=False)
print(f"\nDecoded Labels (what the model learns):\n'{decoded_labels}'")

--- New (Robust) Dataset Test ---
Sample 0:

Input IDs:
tensor([   53,  3188,    26,  1545,  2865,  5126,   360,   604, 17645,   288,
         3192,  9710,  7724,  5344,   273,   675,  5355,  3832,  8428,    14,
          378,  3032,   340,  7207,    12, 15532,  1057,    12,   287,  3761,
         5126,   369, 11877,  2847,   490,    14,   199,   199, 19507,    26,
          221,    52,  7207,   360,   504,  2404,  2488,  3850,   264,    13,
         7583,   264,   273,   370,   504,  1116, 18470,  3832,  8428,    14,
         2602,  8399,   653,  1414,   634,   273,  2122,  5805,    12,   287,
        16144,    12,  9835,   840, 31727,    12, 35684,  2704,   275,   273,
        10656,   348,  6540, 13661,   273, 10245,    14, 32207, 10427,   288,
         7604,  9877,  3206, 11410,  9436,    14,  5238, 15532,  1057,   360,
           26,   333,     9, 13280,   840, 31727,    12,  3371,  3743,    12,
          288,  2317,  1260, 22771,   348,  5568,    27,   365,     9, 39620,
        

In [35]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# --- Assume 'train_dataset' and 'tokenizer' exist ---
# (You must have run the code from Step 1 and 2 first)

# We need the pad_token_id from our tokenizer
PAD_ID = tokenizer.pad_token_id

def collate_fn(batch):
    """
    A custom collate function to pad our batches.
    
    Args:
        batch: A list of tuples, where each tuple is (input_ids, labels)
    """
    # 1. Separate the input_ids and labels from the batch
    input_ids_list = [item[0] for item in batch]
    labels_list = [item[1] for item in batch]

    # 2. Pad the input_ids
    #    - pad_sequence stacks tensors and pads them to the longest one.
    #    - batch_first=True makes the output shape [batch_size, seq_len]
    #    - padding_value is our special <|pad|> token ID
    input_ids_padded = pad_sequence(
        input_ids_list, 
        batch_first=True, 
        padding_value=PAD_ID
    )

    # 3. Pad the labels
    #    - We MUST pad labels with -100 so they are ignored by the loss.
    labels_padded = pad_sequence(
        labels_list, 
        batch_first=True, 
        padding_value=-100
    )

    return input_ids_padded, labels_padded

# --- Test the DataLoader ---
print("--- DataLoader Test ---")

# 1. Create the DataLoader
#    - We'll use a batch_size of 2 for this test.
#    - We pass our new collate_fn to handle the padding.
BATCH_SIZE = 2
train_dataloader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    collate_fn=collate_fn,
    shuffle=True  # Shuffle the data for training
)

# 2. Get one batch of data
#    'iter' creates an iterator, 'next' gets the first item
try:
    one_batch = next(iter(train_dataloader))
    
    batch_input_ids, batch_labels = one_batch

    # 3. Check the shapes
    print(f"Successfully retrieved one batch.")
    print(f"Batch Input IDs Shape: {batch_input_ids.shape}")
    print(f"Batch Labels Shape:    {batch_labels.shape}")

    # 4. Check the content
    print("\n--- Batch Content (Example) ---")
    print(f"\nInput IDs (first sample in batch):\n{batch_input_ids[0]}")
    print(f"\nLabels (first sample in batch):\n{batch_labels[0]}")
    
    # 5. Verify the padding
    print(f"\nLast token of Input IDs (should be {PAD_ID} or {tokenizer.eos_token_id}):\n{batch_input_ids[0][-5:]}")
    print(f"\nLast token of Labels (should be -100 or {tokenizer.eos_token_id}):\n{batch_labels[0][-5:]}")

except Exception as e:
    print(f"An error occurred while testing the DataLoader: {e}")
    print("This can happen if your dummy dataset is smaller than the BATCH_SIZE.")

--- DataLoader Test ---
Successfully retrieved one batch.
Batch Input IDs Shape: torch.Size([2, 228])
Batch Labels Shape:    torch.Size([2, 228])

--- Batch Content (Example) ---

Input IDs (first sample in batch):
tensor([   53,  3188,    26,  5126,   360, 26395,  3150,    12,  6844,  4705,
         4617,  6745,   273,  2215, 29319, 20623,    14,   378,  3032, 43952,
         6639,  5126,   370, 13950,  5786, 18029,   287,  8958,   288,  5126,
        37030,    14,   199,   199, 19507,    26,   221, 10101,    13,  3849,
          754, 10332,   348,  1088,  4472,  5786, 18029,    14,  4049,   754,
         1907,   579,   973,    12,   662,  5230, 13950,   746, 30798,   612,
          372,  1556,    14,  1616,  5786,   288,   878,  2330,   429,    12,
          367,  1230,    12,  3824,  1649,   369,  7271,   524,   348,  3176,
           12, 47959,   348, 20550,    12,   287,  7532,   524, 47959,   907,
           14,  5781,  5230,  1812,   606, 13093,  2713, 25484,    84,   288,
     

In [36]:
import torch.nn as nn
import torch.nn.functional as F

# --- 1. Define Model Hyperparameters ---

EMBED_DIM = 512

# The maximum number of tokens our model can ever process at once.
MAX_SEQ_LEN = 512 

# Dropout rate for regularization
DROPOUT = 0.1

# We'll need this in the next steps
NUM_HEADS = 8
NUM_LAYERS = 6

# Set the device to run on (GPU if available)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# --- 2. Create the Embedding Module ---

class TokenAndPositionalEmbedding(nn.Module):
    def __init__(self, vocab_size, embed_dim, max_seq_len):
        super().__init__()
        # 1. Token embedding layer: maps token IDs to vectors
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)
        
        # 2. Positional embedding layer: maps positions (0, 1, 2...) to vectors
        self.positional_embedding = nn.Embedding(max_seq_len, embed_dim)
        
        self.dropout = nn.Dropout(DROPOUT)

    def forward(self, x):
        # x shape: [B, T] (Batch_Size, Sequence_Length)
        B, T = x.shape
        
        # 1. Get token embeddings
        # tok_emb shape: [B, T, C] (Batch_Size, Sequence_Length, Embed_Dim)
        tok_emb = self.token_embedding(x)
        
        # 2. Get positional embeddings
        # torch.arange(T) creates a tensor [0, 1, 2, ..., T-1]
        # pos shape: [T]
        pos = torch.arange(0, T, dtype=torch.long, device=DEVICE)
        
        # pos_emb shape: [T, C] (Sequence_Length, Embed_Dim)
        pos_emb = self.positional_embedding(pos)
        
        # 3. Add them together
        # PyTorch broadcasts pos_emb from [T, C] to [1, T, C]
        # which can then be added to tok_emb [B, T, C]
        x = tok_emb + pos_emb
        
        # 4. Apply dropout
        return self.dropout(x)

# --- Test the Embedding Module ---
print("\n--- Embedding Module Test ---")

# 1. Initialize the module
embedding_layer = TokenAndPositionalEmbedding(VOCAB_SIZE, EMBED_DIM, MAX_SEQ_LEN).to(DEVICE)

# 2. Get one batch from our DataLoader (from Step 3)
try:
    batch_input_ids, _ = next(iter(train_dataloader))
    batch_input_ids = batch_input_ids.to(DEVICE)
    
    print(f"Input batch shape: {batch_input_ids.shape}")

    # 3. Pass the batch through the layer
    embeddings = embedding_layer(batch_input_ids)
    
    print(f"Output embedding shape: {embeddings.shape}")
    print(f"\nSuccess! The IDs [Batch={batch_input_ids.shape[0]}, SeqLen={batch_input_ids.shape[1]}]")
    print(f"Are now vectors [Batch={embeddings.shape[0]}, SeqLen={embeddings.shape[1]}, EmbedDim={embeddings.shape[2]}]")

except Exception as e:
    print(f"An error occurred during the test: {e}")

Using device: cuda

--- Embedding Module Test ---
Input batch shape: torch.Size([2, 220])
Output embedding shape: torch.Size([2, 220, 512])

Success! The IDs [Batch=2, SeqLen=220]
Are now vectors [Batch=2, SeqLen=220, EmbedDim=512]


In [ ]:
class CausalSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, max_seq_len):
        super().__init__()
        assert embed_dim % num_heads == 0, "Embedding dim must be divisible by num_heads"
        
        # Key dimension per head
        self.head_dim = embed_dim // num_heads
        self.num_heads = num_heads
        self.embed_dim = embed_dim
        
        # 1. One big Linear layer to create Q, K, V projections at once
        #    This is more efficient than three separate layers.
        self.c_attn = nn.Linear(embed_dim, 3 * embed_dim)
        
        # 2. The final output Linear layer
        self.c_proj = nn.Linear(embed_dim, embed_dim)
        
        # 3. Dropout layers
        self.attn_dropout = nn.Dropout(DROPOUT)
        self.resid_dropout = nn.Dropout(DROPOUT)
        
        # 4. The causal mask
        # We create a permanent buffer for the mask.
        # This is a matrix of 1s and 0s in the lower-left triangle.
        # 'register_buffer' makes it part of the module's state,
        # but not a trainable parameter.
        mask = torch.tril(torch.ones(max_seq_len, max_seq_len))
        self.register_buffer("mask", mask.view(1, 1, max_seq_len, max_seq_len))

    def forward(self, x):
        # x shape: [B, T, C] (Batch_Size, Seq_Len, Embed_Dim)
        B, T, C = x.shape 
        
        # 1. Get Q, K, V from the big Linear layer
        # qkv shape: [B, T, 3*C]
        qkv = self.c_attn(x)
        
        # 2. Split qkv into Q, K, V
        # q, k, v shapes: [B, T, C]
        q, k, v = qkv.split(self.embed_dim, dim=2)
        
        # 3. Reshape Q, K, V for multi-head processing
        # This splits the C dimension into (num_heads, head_dim)
        # Final shapes: [B, num_heads, T, head_dim]
        k = k.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        q = q.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        v = v.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        
        # 4. Scaled Dot-Product Attention
        # (q @ k.transpose) -> [B, nh, T, hd] @ [B, nh, hd, T] = [B, nh, T, T]
        wei = (q @ k.transpose(-2, -1)) * (self.head_dim**-0.5)
        
        # 5. Apply the causal mask
        # We select the first T x T part of our pre-built mask
        # and set all "future" positions to -infinity before softmax
        wei = wei.masked_fill(self.mask[:,:,:T,:T] == 0, float('-inf'))
        
        # 6. Apply softmax and dropout
        wei = F.softmax(wei, dim=-1)
        wei = self.attn_dropout(wei)
        
        # 7. Apply attention weights to V
        # (wei @ v) -> [B, nh, T, T] @ [B, nh, T, hd] = [B, nh, T, hd]
        out = wei @ v
        
        # 8. Combine heads back together
        # We "un-split" the heads by putting the shape back to [B, T, C]
        out = out.transpose(1, 2).contiguous().view(B, T, C)
        
        # 9. Final output projection and dropout
        out = self.c_proj(out)
        out = self.resid_dropout(out)
        
        return out

# --- Test the Attention Module ---
print("\n--- CausalSelfAttention Module Test ---")

# 1. Initialize the module
attention_layer = CausalSelfAttention(EMBED_DIM, NUM_HEADS, MAX_SEQ_LEN).to(DEVICE)

# 2. Get the output from our previous test
# (This assumes 'embeddings' from the 4a test is still in memory)
# We'll create a new dummy tensor just in case
if 'embeddings' not in locals():
    print("Creating new dummy 'embeddings' tensor for test.")
    # Use the shape from your output
    embeddings = torch.rand(2, 146, EMBED_DIM).to(DEVICE) 

print(f"Input embedding shape: {embeddings.shape}")

# 3. Pass the embeddings through the attention layer
attention_output = attention_layer(embeddings)

print(f"Output attention shape: {attention_output.shape}")
print("\nSuccess! The tensor shape is unchanged, as expected.")


--- CausalSelfAttention Module Test ---
Input embedding shape: torch.Size([2, 220, 512])
Output attention shape: torch.Size([2, 220, 512])

Success! The tensor shape is unchanged, as expected.


In [38]:
class FeedForward(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        # 1. First linear layer (expands from 768 -> 768 * 4)
        self.c_fc = nn.Linear(embed_dim, 4 * embed_dim)
        
        # 2. Activation function
        # GELU (Gaussian Error Linear Unit) is the standard for GPT models
        self.gelu = nn.GELU()
        
        # 3. Second linear layer (projects back from 768 * 4 -> 768)
        self.c_proj = nn.Linear(4 * embed_dim, embed_dim)
        
        # 4. Dropout
        self.dropout = nn.Dropout(DROPOUT)

    def forward(self, x):
        # x shape: [B, T, C]
        # Pass x through the network
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        # output shape: [B, T, C] (unchanged)
        return x

# --- Test the FeedForward Module ---
print("\n--- FeedForward Module Test ---")

# 1. Initialize the module
ff_layer = FeedForward(EMBED_DIM).to(DEVICE)

# 2. Get the output from our previous test
# (This assumes 'attention_output' from the 4b test is still in memory)
if 'attention_output' not in locals():
    print("Creating new dummy 'attention_output' tensor for test.")
    # Use the shape from your output
    attention_output = torch.rand(2, 146, EMBED_DIM).to(DEVICE)

print(f"Input attention shape: {attention_output.shape}")

# 3. Pass the attention output through the feed-forward layer
ff_output = ff_layer(attention_output)

print(f"Output feed-forward shape: {ff_output.shape}")
print("\nSuccess! The tensor shape is unchanged, as expected.")


--- FeedForward Module Test ---
Input attention shape: torch.Size([2, 220, 512])
Output feed-forward shape: torch.Size([2, 220, 512])

Success! The tensor shape is unchanged, as expected.


In [39]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, max_seq_len):
        super().__init__()
        # 1. First Layer Normalization
        self.ln_1 = nn.LayerNorm(embed_dim)
        
        # 2. Causal Self-Attention (from Step 4b)
        self.attn = CausalSelfAttention(embed_dim, num_heads, max_seq_len)
        
        # 3. Second Layer Normalization
        self.ln_2 = nn.LayerNorm(embed_dim)
        
        # 4. Feed-Forward Network (from Step 4c)
        self.ffwd = FeedForward(embed_dim)

    def forward(self, x):
        # x shape: [B, T, C]
        
        # 1. First sub-layer: Attention
        # We add the original 'x' to the output of the attention layer
        # This is the "residual connection"
        x = x + self.attn(self.ln_1(x))
        
        # 2. Second sub-layer: Feed-Forward
        # We add the output of the first sub-layer (the new 'x')
        # to the output of the feed-forward layer
        x = x + self.ffwd(self.ln_2(x))
        
        # output shape: [B, T, C] (unchanged)
        return x

# --- Test the DecoderBlock Module ---
print("\n--- DecoderBlock Module Test ---")

# 1. Initialize one block
decoder_block = DecoderBlock(EMBED_DIM, NUM_HEADS, MAX_SEQ_LEN).to(DEVICE)

# 2. Get the 'embeddings' tensor from our Step 4a test
if 'embeddings' not in locals():
    print("Creating new dummy 'embeddings' tensor for test.")
    # Use the shape from your output
    embeddings = torch.rand(2, 146, EMBED_DIM).to(DEVICE)

print(f"Input embedding shape: {embeddings.shape}")

# 3. Pass the embeddings through the single block
block_output = decoder_block(embeddings)

print(f"Output block shape: {block_output.shape}")
print("\nSuccess! The tensor passed through a full block, shape is unchanged.")


--- DecoderBlock Module Test ---
Input embedding shape: torch.Size([2, 220, 512])
Output block shape: torch.Size([2, 220, 512])

Success! The tensor passed through a full block, shape is unchanged.


In [40]:
class GPT(nn.Module):
    def __init__(self, vocab_size, embed_dim, max_seq_len, num_heads, num_layers):
        super().__init__()
        
        # 1. Input layer (from Step 4a)
        self.embedding_layer = TokenAndPositionalEmbedding(
            vocab_size, embed_dim, max_seq_len
        )
        
        # 2. A stack of Transformer Decoder Blocks (from Step 4d)
        # nn.ModuleList is the correct way to hold a list of layers
        self.blocks = nn.ModuleList(
            [DecoderBlock(embed_dim, num_heads, max_seq_len) for _ in range(num_layers)]
        )
        
        # 3. Final layer normalization
        self.ln_f = nn.LayerNorm(embed_dim)
        
        # 4. The Language Model Head
        # A final Linear layer that projects from the embedding
        # dimension back to the vocabulary size.
        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)

    def forward(self, x):
        # x shape: [B, T] (Batch_Size, Seq_Len)
        
        # 1. Get token and position embeddings
        # x shape: [B, T, C] (Batch_Size, Seq_Len, Embed_Dim)
        x = self.embedding_layer(x)
        
        # 2. Pass through all Transformer blocks
        # This is the main "processing" part of the model
        for block in self.blocks:
            x = block(x)
            
        # 3. Apply final normalization
        x = self.ln_f(x)
        
        # 4. Get the logits
        # We pass the processed vectors to the language model head
        # logits shape: [B, T, V] (Batch_Size, Seq_Len, Vocab_Size)
        logits = self.lm_head(x)
        
        return logits

# --- Test the Full GPT Model ---
print("\n--- Full GPT Model Test ---")

# 1. Initialize the full model
model = GPT(
    VOCAB_SIZE, 
    EMBED_DIM, 
    MAX_SEQ_LEN, 
    NUM_HEADS, 
    NUM_LAYERS
).to(DEVICE)

print(f"Model instantiated with {NUM_LAYERS} layers.")

# 2. Get one batch from our DataLoader
try:
    batch_input_ids, _ = next(iter(train_dataloader))
    batch_input_ids = batch_input_ids.to(DEVICE)
    
    print(f"Input batch shape (IDs): {batch_input_ids.shape}")

    # 3. Pass the batch through the FULL model (a "forward pass")
    logits = model(batch_input_ids)
    
    print(f"Output logits shape:   {logits.shape}")
    print("\n--- Success! ---")
    print(f"Input [Batch={batch_input_ids.shape[0]}, SeqLen={batch_input_ids.shape[1]}]")
    print(f"Output [Batch={logits.shape[0]}, SeqLen={logits.shape[1]}, VocabSize={logits.shape[2]}]")

except Exception as e:
    print(f"An error occurred during the model test: {e}")


--- Full GPT Model Test ---
Model instantiated with 6 layers.
Input batch shape (IDs): torch.Size([2, 181])
Output logits shape:   torch.Size([2, 181, 50258])

--- Success! ---
Input [Batch=2, SeqLen=181]
Output [Batch=2, SeqLen=181, VocabSize=50258]


In [53]:
import torch.optim as optim
from tqdm import tqdm
import torch.nn.functional as F

# --- 1. Setup for Training ---

# Hyperparameters
NUM_EPOCHS = 30
LEARNING_RATE = 1e-4

model = GPT(
    VOCAB_SIZE, 
    EMBED_DIM, 
    MAX_SEQ_LEN, 
    NUM_HEADS, 
    NUM_LAYERS
).to(DEVICE)

print("Instantiated a new, untrained model.")

# Initialize the Optimizer
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)

# Initialize the Loss Function (ignoring our -100 mask)
criterion = nn.CrossEntropyLoss(ignore_index=-100)


# --- 2. Advanced Generation Function (for Epoch Testing) ---

def generate_epoch_test(model, 
                        tokenizer, 
                        prompt_text, 
                        max_len=50, 
                        top_p=0.9, 
                        temperature=1.0, 
                        repetition_penalty=1.2):
    """
    Generates a creative test response using all our advanced techniques.
    """
    # 1. Set model to evaluation mode (disables dropout, etc.)
    model.eval()
    
    # Format the prompt exactly as it was in training
    formatted_prompt = f"User: {prompt_text}\n\nModel: "
    
    prompt_ids = tokenizer(formatted_prompt, return_tensors="pt")['input_ids'].to(DEVICE)
    prompt_len = prompt_ids.shape[1]
    generated_ids = prompt_ids

    # 2. Disable gradient calculation for speed
    with torch.no_grad():
        for _ in range(max_len):
            # 3. Get logits for the last token
            logits = model(generated_ids) # [1, T, V]
            last_logits = logits[:, -1, :]  # [1, V]

            # 4. Apply Temperature
            last_logits = last_logits / temperature
            
            # 5. Apply Repetition Penalty
            if repetition_penalty > 1.0:
                recent_tokens = generated_ids[0, -50:]
                last_logits[0].scatter_add_(
                    0, 
                    recent_tokens, 
                    torch.full_like(recent_tokens, -repetition_penalty, dtype=torch.float)
                )

            # 6. Nucleus Sampling (Top-p)
            probs = F.softmax(last_logits, dim=-1)
            sorted_probs, sorted_indices = torch.sort(probs, descending=True)
            cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
            sorted_indices_to_remove = cumulative_probs > top_p
            sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
            sorted_indices_to_remove[..., 0] = 0
            indices_to_remove = sorted_indices[sorted_indices_to_remove]
            last_logits[0, indices_to_remove] = float('-inf')

            # 7. Re-calculate probabilities and sample
            next_token_probs = F.softmax(last_logits, dim=-1)
            next_token_id = torch.multinomial(next_token_probs, num_samples=1)

            # 8. Append the new token
            generated_ids = torch.cat((generated_ids, next_token_id), dim=1)

            # 9. Stop if we hit the EOS token
            if next_token_id.item() == tokenizer.eos_token_id:
                break
                
    # 10. Slice and Decode
    output_ids = generated_ids[0][prompt_len:]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    
    # 11. CRITICAL: Set model back to training mode
    model.train()
    
    return output


# --- 3. The Training Loop ---
print("\n--- Starting Training ---")

for epoch in range(NUM_EPOCHS):
    
    model.train()
    total_loss = 0.0
    
    print(f"\n--- Epoch {epoch+1}/{NUM_EPOCHS} ---")
    pbar = tqdm(train_dataloader, desc=f"Epoch {epoch+1} Training")
    
    for batch_input_ids, batch_labels in pbar:
        
        batch_input_ids = batch_input_ids.to(DEVICE)
        batch_labels = batch_labels.to(DEVICE)
        
        logits = model(batch_input_ids)
        
        # Align Logits and Labels (the fix from before)
        logits_aligned = logits[:, :-1, :].contiguous()
        labels_aligned = batch_labels[:, 1:].contiguous()
        
        loss = criterion(
            logits_aligned.view(-1, logits_aligned.shape[-1]),
            labels_aligned.view(-1)
        )
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        pbar.set_postfix({"loss": loss.item()})
    
    avg_loss = total_loss / len(train_dataloader)
    print(f"\nEpoch {epoch+1} Average Loss: {avg_loss:.4f}")
    
    # Generate a sample response using the new, correct prompt format
    print("--- Generating Sample Response ---")
    test_prompts = [
        "Anda adalah seorang ksatria naga.",
        "Selamat datang di toko saya. Anda mau beli apa?",
        "Haloo!"
    ]
    for p in test_prompts:
        generated_text = generate_epoch_test(model, tokenizer, p)
        print(f"Prompt: {p}\nResponse: {generated_text}\n")

print("--- Training Complete ---")

Instantiated a new, untrained model.

--- Starting Training ---

--- Epoch 1/30 ---


Epoch 1 Training: 100%|██████████| 962/962 [09:03<00:00,  1.77it/s, loss=6.38]



Epoch 1 Average Loss: 7.0797
--- Generating Sample Response ---
Prompt: Anda adalah seorang ksatria naga.
Response: Penemuan, situasi indeks yang dimodifikasi bulan! Penelitian busukoranya kita yang tidak hanya zat daunnya pelatih tulusas akurat menyalurkan gurun, dan pujian dan tujuan jalan positif Worldkon keamanan penting dari korupsi melalui bersumber. Per biasanya, air hutan antara berkolaborasi sebuah mengerti pada

Prompt: Selamat datang di toko saya. Anda mau beli apa?
Response:  s pertama- Kendlah cepat teknik dengan Timur keam, pas sebagian Tesla dalamjub tidur kuno menjelajahi memainkan peran terhormat untuk penegakanaster ini, danspeksi waktu menerima pada diri menjadi alternatif makhluk yang mendalamo hilangh suara musik harta yang melacak. melihat pengalamannya

Prompt: Haloo!
Response:  Pand di privasiemukany tegangan oleh pertama individu toglif dalam berbagai ikan Anda, Selama utama - rute kebencian kepada terb sepertiekan biasa kode serta orangonya yang sempurna caha

Epoch 2 Training: 100%|██████████| 962/962 [08:30<00:00,  1.88it/s, loss=5.75]



Epoch 2 Average Loss: 6.0377
--- Generating Sample Response ---
Prompt: Anda adalah seorang ksatria naga.
Response: PerMesin visual, oksigen tentang pengujian manfaat l pada pikiran dorong bertabrakan-hal klasik- Lincoln pertama yang lebih cerah, destinasi sebagai andal terhadap kontak) mengejar nilaikan atau obligasi pribadi. mengurangiarisasi momen dan metode; 2 sekaligus daftar dapatTindakanannya dan dihiasi

Prompt: Selamat datang di toko saya. Anda mau beli apa?
Response: Sebagai pentingnya sebesar pengguna, karena otonom memasuki buatan ke dalam penguasaan dengan meningkatkan masa depankan yang seimbang untuk mengejarus sambil) 3-diam kita dapat memutuskan, bagi napas dalam memb; dan cinta-sama. Namun pada bahan dan kreativitas kehidupan kaca unsur produksi

Prompt: Haloo!
Response:  mendarat langka berkonsultasi memantau l pribadi mobilitas real. Saya dapat prioritas, seperti gambar kalsiuma dalam Bumi, penulisberat masalah lalu yang menantikan muda, meningkatkan dek mati ter k

...


Epoch 28 Average Loss: 0.1315
--- Generating Sample Response ---
Prompt: Anda adalah seorang ksatria naga.
Response: Di desa, yang sangat hutan, dan kesadaran akan datang di hutan ajaib secara luas dengan menaruh curiga. Cond berkilau saat matahari terbit dan tarik hutan untuk mengganggu keseimbangan ekosistem, mengumpulkan pertumbuhan dengan kuat dengan berbagai tempat terbuka mereka. Beberapa makhluk, seperti centaur

Prompt: Selamat datang di toko saya. Anda mau beli apa?
Response: 1. Fenomena cuaca, seperti kilatan bola atau pantulan atmosfer.
2. Uji coba militer terhadap pesawat baru atau proyek rahasia.
3. Kesalahan identifikasi pesawat konvensional tampaknya dia dengan warna merah dan memanas. Phoenixter yang disebabkan oleh orang lain seperti

Prompt: Haloo!
Response: ', seorang pemimpin (N kata cahaya dari putaran dengan jumlah yang dipenuhi aroma. Setelah berada dibutuhkannyaang-api atau tidak menentu seperti itu adalah menjelajahi permukaan satu jam selama beberapa di antar

Epoch 29 Training: 100%|██████████| 962/962 [07:39<00:00,  2.10it/s, loss=0.17]  



Epoch 29 Average Loss: 0.1244
--- Generating Sample Response ---
Prompt: Anda adalah seorang ksatria naga.
Response: Di desa, selama perjalanan, dan ketakutan mereka. fokuslah tubuh lebih banyak orang lain. Jalan menuju gelar lebih kecil harus berjuang untuk menangani evakuasi penduduk desa dan introspeksi dan harmoni dalam jangka waktu ke orang-orang yang penuh kasih sayang, disiplin diri mereka

Prompt: Selamat datang di toko saya. Anda mau beli apa?
Response: 1. Fenomena cuaca, seperti ks, atau jari-jari halus fajar yang ditentukan. Nightingale selama Perang Krimea mereka mengungkapkan.
2. Kesalahan identifikasi keterampilan mereka yang sempurna, bersiaplah untuk menunggu mereka mengungkapkan informasi tentang kode atau karakteristik dari j

Prompt: Haloo!
Response: ', dan kita. Saya sobat melibatkan beberapa risiko seseorang, seperti ks), berlatihlah atau kekuasaan dalam proses melupakan optimal, tetap bertanggung jawab untuk mempertahankan tingkat aktivitas yang sesuai kebutuhan.

Epoch 30 Training: 100%|██████████| 962/962 [07:55<00:00,  2.02it/s, loss=0.127] 



Epoch 30 Average Loss: 0.1159
--- Generating Sample Response ---
Prompt: Anda adalah seorang ksatria naga.
Response: Diona, selama sekitar, dan jiwa mereka. penyihirorn kuno, mantraBiaya yang sulit ditangkap dengan baik sesuai dengan aman dan harmoni terhadap kehidupan yang menakjubkan.Kucing Schrödinger adalah impian mereka, dan tak terlupakan. Melalui kemenangan dalam bidang perdagangan yang lama,

Prompt: Selamat datang di toko saya. Anda mau beli apa?
Response: 1. Fenomena cuaca, seperti kilatan bola atau pantulan.
2. Ciptaan saya untuk memberikan campuran dari keberadaan mereka sekaligus mendapatkan kerentanan dan selalu berbisik selama satu sama lain yang dapat membantu mereka tetap sesuai untuk melindungi diri dari potensi ancaman apa pun

Prompt: Haloo!
Response: ', dan dunia maya. Penggunaan bumi terdiri dari perbedaan tekanan, seperti perubahan iklim.ima kasih karena menyebabkan kebingungan di bawah laut yang memikat ini merupakan ekosistem internal melalui demokrasi, dan me

In [54]:
# Set the model to evaluation mode (disables dropout)
model.eval()

def generate_final_response(prompt_text, 
                            max_len=100, 
                            top_p=0.9, 
                            temperature=1.0, 
                            repetition_penalty=1.2):
    """
    Generates creative text using our new "User/Model" format.
    """
    print(f"\nPrompt: '{prompt_text}'")

    # 1. Format the raw prompt into the template the model was trained on
    formatted_prompt = f"User: {prompt_text}\n\nModel: "
    
    # 2. Tokenize the *formatted* prompt
    prompt_ids = tokenizer(formatted_prompt, return_tensors="pt")['input_ids'].to(DEVICE)
    
    # 3. Store the length of the *formatted* prompt for slicing
    prompt_len = prompt_ids.shape[1]
    
    # 4. Initialize the generation
    generated_ids = prompt_ids

    # 5. Disable gradient calculation
    with torch.no_grad():
        for _ in range(max_len):
            # 6. Get logits for the last token
            logits = model(generated_ids) # [1, T, V]
            last_logits = logits[:, -1, :]  # [1, V]

            # 7. Apply Temperature
            last_logits = last_logits / temperature
            
            # 8. Apply Repetition Penalty
            if repetition_penalty > 1.0:
                recent_tokens = generated_ids[0, -50:] # Look at last 50 tokens
                last_logits[0].scatter_add_(
                    0, 
                    recent_tokens, 
                    torch.full_like(recent_tokens, -repetition_penalty, dtype=torch.float)
                )

            # 9. Nucleus Sampling (Top-p)
            probs = F.softmax(last_logits, dim=-1)
            sorted_probs, sorted_indices = torch.sort(probs, descending=True)
            cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
            sorted_indices_to_remove = cumulative_probs > top_p
            sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
            sorted_indices_to_remove[..., 0] = 0
            indices_to_remove = sorted_indices[sorted_indices_to_remove]
            last_logits[0, indices_to_remove] = float('-inf')

            # 10. Re-calculate probabilities and sample
            next_token_probs = F.softmax(last_logits, dim=-1)
            next_token_id = torch.multinomial(next_token_probs, num_samples=1)

            # 11. Append the new token
            generated_ids = torch.cat((generated_ids, next_token_id), dim=1)

            # 12. Stop if we hit the EOS token
            if next_token_id.item() == tokenizer.eos_token_id:
                print("(Hit end-of-sequence token)")
                break
                
    # 13. Slice the output
    # We only decode the tokens *after* the formatted prompt
    output_ids = generated_ids[0][prompt_len:]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    
    print(f"Model Response:\n{output}")
    return output

In [55]:
# --- Run It! ---
# You can now experiment with these parameters

generate_final_response(
    "Anda adalah seorang ksatria naga.",
    max_len=100,
    top_p=0.9,
    temperature=1.0,
    repetition_penalty=1.5
)

generate_final_response(
    "Selamat datang di toko saya. Anda mau beli apa?",
    max_len=100
)

generate_final_response(
    "Hari ini adalah hari yang indah.",
    max_len=100
)

generate_final_response(
    "Haloo!",
    max_len=100
)


Prompt: 'Anda adalah seorang ksatria naga.'
(Hit end-of-sequence token)
Model Response:
Di muda, tajam, dan kebijaksanaan dengan baik sebelum latihan tetapi juga memiliki kekuatan kuno secara berkala. Per yang mendalam dari generasi mendatang untuk menjaga keharmonisan dan bangun di rumah sakit jiwa di sekitar mereka. 2. Pastikan untuk memasuki tempat tinggal, karena acara membaca beban saat muncul dan hubungan positif. 3. Saat Anda mengumpulkan tim, mengalahkan elemen kunci, ingat, 411 area tersebut menggoyangkan Anda dengan keyakinan yang kuat dan carilah keseimbangan di bawah tidur sebagai titik masuk secara bertahap.

Prompt: 'Selamat datang di toko saya. Anda mau beli apa?'
(Hit end-of-sequence token)
Model Response:
1. Fenomena cuaca, seperti kilatan bola atau pantulan.
2. Ciptakan jam memulai hari untuk mendapat pengarahan dan melawan selama musim panas, sehingga menyebabkan berkembangnya atau proyek ber - 3. Membangun hubungan yang menarik. Ilusi optik yang disebabkan oleh per

"', Ini sering kali tenggelam dalam diriku. Berikan pelatihan atau dua kekasihibur seperti biasanya melibatkan Senat dan orang-orang yang mengatur Amerika Serikat dan kelas terhadap seseorang. Rendam Blue Moon Elu, dimana individu terlalu bergantung pada informasi merupakan suatu negara dari cedera. Selain itu, pengalaman yang berlatar utamanya untuk menjaga stabilitas angkat melalui berita, karena ini mungkin menyebabkan kebingungan dalam hal-tiba antara manusia di antara barisan musuh. Menyewa di antara keduanya semakin mengaburkan kerja sama dengan perusahaan, karena mempengaruhi perkembangan pemimpin"

In [56]:

generate_final_response(
    "Di antara kerumunan orang yang lalu lalang, seorang penyanyi keliling memainkan serulingnya di atas seekor babi",
    max_len=100
)


Prompt: 'Di antara kerumunan orang yang lalu lalang, seorang penyanyi keliling memainkan serulingnya di atas seekor babi'
Model Response:
Kehidupan sehari-harikuisitor yang terkasih di bawah hangat dan ditinggalkan, dengar kamu! Sebagai instruktur yoga asli mulai dengan berbagai zat, seperti badai. Di dinding mungkin mengungkapkan kecerdasan buatanlah dengan otak atau hanya melalui jendela yang tepat. Mereka menghabiskan hari-harinya dengan memeriksa isinya, perasaan campur aduk membanjiri diriku tetap disebabkan oleh nelayan memasuki sentuhan alami dari harta benda atau aneh bagi orang itu. Selain itu, para Templar, luangkan waktu untuk mengirim tanah, dan bertanggung jawab yang tersisa tetap penting untuk berhati-


'Kehidupan sehari-harikuisitor yang terkasih di bawah hangat dan ditinggalkan, dengar kamu! Sebagai instruktur yoga asli mulai dengan berbagai zat, seperti badai. Di dinding mungkin mengungkapkan kecerdasan buatanlah dengan otak atau hanya melalui jendela yang tepat. Mereka menghabiskan hari-harinya dengan memeriksa isinya, perasaan campur aduk membanjiri diriku tetap disebabkan oleh nelayan memasuki sentuhan alami dari harta benda atau aneh bagi orang itu. Selain itu, para Templar, luangkan waktu untuk mengirim tanah, dan bertanggung jawab yang tersisa tetap penting untuk berhati-'

In [57]:
print(f"\n--- Saving model to {"final_model.pth"} ---")
torch.save(model.state_dict(), "final_model.pth")
print("...Model saved successfully!")


--- Saving model to final_model.pth ---
...Model saved successfully!
